In [104]:
from collections import deque
import pandas as pd
import numpy as np

In [105]:
from haversine import haversine

In [106]:
import math

In [107]:
from numba import jit
from time import perf_counter

In [108]:
import folium

In [109]:
node = pd.read_csv('/Users/juhong/Library/CloudStorage/OneDrive-아주대학교/대학교/3학년 2학기/캡스톤/DUI-traffic-stop-recommendation/Optimal_Choice_Algorithm/gangnam_node_2019.csv',encoding = 'cp949')

In [110]:
node.head()

,NODE_ID,NODE_TYPE,NODE_NAME,y좌표,x좌표
0,1220001100,108,동부간선도로-남부순환,127.090548,37.496133
1,1220000700,108,수서IC2,127.103633,37.491131
2,1220001500,108,동부간선도로-남부순환,127.077407,37.500554
3,1220025600,108,수서IC6,127.095864,37.492109
4,1220058500,108,올림픽대로-일반국도47,127.051645,37.528632


In [111]:
link = pd.read_csv('/Users/juhong/Library/CloudStorage/OneDrive-아주대학교/대학교/3학년 2학기/캡스톤/DUI-traffic-stop-recommendation/Optimal_Choice_Algorithm/gangnam_link_final.csv',encoding = 'cp949')
link2 = pd.read_csv('/Users/juhong/Library/CloudStorage/OneDrive-아주대학교/대학교/3학년 2학기/캡스톤/DUI-traffic-stop-recommendation/Optimal_Choice_Algorithm/link_data_all.csv',encoding = 'cp949')

In [112]:
link.head()

,LINK_ID,F_NODE,T_NODE,ROAD_NAME,TRAFFIC,LANES,MAX_SPD
0,1220110900,1220063900,1220060300,선릉로189길,0,1,30
1,1220111000,1220060300,1220063900,선릉로189길,0,1,30
2,1220109700,1220035000,1220035500,선릉로89길,0,1,60
3,1220110500,1220045700,1220046500,봉은사로113길,0,1,40
4,1220110300,1220044800,1220045700,봉은사로113길,0,2,40


In [113]:
#dictionary를 활용하여 graph를 만들기 위한 sorting
node_sorted = node.sort_values(by=['NODE_ID'])

In [114]:
#node_reset_index
node_sorted = node_sorted.reset_index()
node_sorted

,index,NODE_ID,NODE_TYPE,NODE_NAME,y좌표,x좌표
0,535,1210012800,101,서초구-강남구경계(2),127.090283,37.463452
1,509,1210015200,105,서초구-강남구경계(3),127.065345,37.469391
2,528,1210016400,101,국악고교앞,127.048734,37.473734
3,517,1210017100,101,두상빌딩,127.046537,37.475846
4,529,1210019000,101,포이사거리,127.044592,37.478301
...,...,...,...,...,...,...
536,505,1220148500,101,율현2빗물펌프장,127.119206,37.471227
537,506,1220150600,104,구룡지하차도(동측),127.055029,37.474024
538,540,1230112800,105,서울시-성남시경계,127.127243,37.468450
539,538,2040015700,105,-,127.107924,37.462214


In [115]:
# index = node_sorted['NODE_ID'].searchsorted(1220000700)
# index
node.shape[0]

541

In [116]:
#dictionary를 활용한 undirected graph, 0과 651확인 필
graph = {}

check = [[0 for i in range(node.shape[0]+1)] for j in range(node.shape[0]+1)]

for i,data in link2.iterrows():
    start,end = data['F_NODE'],data['T_NODE']
    index_start = node_sorted['NODE_ID'].searchsorted(start)
    index_end = node_sorted['NODE_ID'].searchsorted(end)
    
    #link의 시작노드, 끝노드 명을 index로 변환
    link['F_NODE'][i] = index_start
    link['T_NODE'][i] = index_end
    link2['F_NODE'][i] = index_start
    link2['T_NODE'][i] = index_end
    
    if(not check[index_start][index_end]):
        graph[index_start] = graph.get(index_start,[]) + [index_end]
        graph[index_end] = graph.get(index_end,[]) + [index_start]
    
    check[index_start][index_end] = 1
    check[index_end][index_start] = 1

graph

/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['F_NODE'][i] = index_start
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['T_NODE'][i] = index_end
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['F_NODE'][i] = index_start
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['T_NODE'][i] = index_end
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['T_NODE'][i] = index_end
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link2['F_NODE'][i] = index_start
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link2['T_NODE'][i] = index_end
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['F_NODE'][i] = index_start
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link2['F_NODE'][i] = index_start
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link2['T_NODE'][i] = index_end
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

...

/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link2['T_NODE'][i] = index_end
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['F_NODE'][i] = index_start
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['T_NODE'][i] = index_end
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link2['F_NODE'][i] = index_start
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['F_NODE'][i] = index_start
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['T_NODE'][i] = index_end
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link2['F_NODE'][i] = index_start
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link2['T_NODE'][i] = index_end
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/548121775.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

{526: [494],
 494: [526, 493],
 281: [286, 276, 288],
 286: [281, 290, 277],
 372: [378, 363, 359],
 378: [372, 389, 365, 391],
 363: [372, 352, 365, 353],
 352: [363, 340, 330],
 26: [27],
 27: [26, 416, 414, 25],
 416: [27, 44, 28],
 527: [425, 403, 439],
 425: [527, 429, 434, 444, 403],
 247: [37, 64, 245, 524],
 37: [247, 39, 524],
 414: [27, 419],
 531: [147, 176],
 147: [531, 141, 176],
 532: [165, 133, 154],
 165: [532, 174, 133, 154],
 529: [122, 33, 140],
 122: [529, 530, 108, 142],
 530: [122, 33, 118],
 108: [122, 128, 32],
 403: [527, 359, 398, 425, 66],
 44: [416, 428, 435],
 92: [94, 1],
 94: [92, 97, 100],
 36: [524, 224, 38],
 524: [36, 219, 523, 37, 247],
 196: [197, 202, 56],
 197: [196, 34, 188],
 224: [36, 209, 216],
 464: [451, 472, 476],
 451: [464, 444, 445],
 463: [472, 456, 444],
 472: [463, 464],
 488: [481, 505, 507],
 481: [488, 489, 468, 478, 480],
 489: [481, 504, 499],
 504: [489, 45, 509],
 505: [488, 496, 45],
 514: [512, 511],
 512: [514, 45, 510],
 51

In [117]:
class Cluster:
    def __init__(self,coordinate_center,radius):
        self.center = coordinate_center
        self.radius = radius
    
    def in_distance(self,coordinate):
        if(haversine(self.center,coordinate)*1000<=self.radius):
            return True
        else:
            return False

In [118]:
df_cluster = pd.read_csv('/Users/juhong/Library/CloudStorage/OneDrive-아주대학교/대학교/3학년 2학기/캡스톤/DUI-traffic-stop-recommendation/Optimal_Choice_Algorithm/cluster_data.csv',encoding = 'cp949')

set_cluster = [0 for i in range(df_cluster.shape[0])]

for i,data in df_cluster.iterrows():
    set_cluster[i] = Cluster((data['x좌표'],data['y좌표']),data['radius'])
    

In [119]:
def BFS(graph,start,dist):
    
    queue = deque([start])
    
    while queue:
        now = queue.popleft()
        
        for i in graph[now]:
            if(dist[i]==-1):
                queue.append(i)
                dist[i] = dist[now]+1
    return

In [120]:
short_check = [[0 for i in range(node.shape[0]+1)] for j in range(node.shape[0]+1)]

In [121]:
for i,data in link2.iterrows():
    #destination 즉, 의도하는 거리보다 작은 거리에 위치하는 node
    dest = deque()
    #거리를 저장
    dist = [-1 for i in range(node.shape[0])]
    
    start_s = data['F_NODE']
    start_e = data['T_NODE']
    
    #시작노드 외 나머지 node는 거리가 -1로 시작
    try:
        dist[start_s] = 0
        BFS(graph,start_s,dist)
    except:
        print(start_s)
        print(data['F_NODE'])
        print(i+1)
        
        break
    
    print(dist)
    
    for i in range(node.shape[0]):
        if(dist[i] <10):
            dest.append((start_s,i))

    dist = [-1 for i in range(node.shape[0])]
    dist[start_e] = 0

    BFS(graph,start_e,dist)
    
    for i in range(node.shape[0]):
        if(dist[i] <10):
            dest.append((start_e,i))
    
    dest = list(set(dest))
    
    for v in dest:
        short_check[v[0]][v[1]] = True
        short_check[v[1]][v[0]] = True

[23, 24, 24, 25, 24, 23, 23, 24, 24, 23, 23, 23, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 14, 13, 13, 14, 14, 13, 13, 13, -1, 23, 20, 19, 14, 17, 13, 11, 12, 10, 16, 11, 8, 10, 11, 8, 24, 22, 21, 21, 19, 22, 19, 16, 19, 17, 15, 16, 17, 21, 15, 15, 19, 16, 13, 20, 10, 9, -1, 24, 24, 22, 22, 24, 24, 23, 23, 23, 22, 22, 22, 22, 25, 25, 25, 24, 25, 24, 23, 23, 21, 20, 23, 25, 22, 24, 19, 21, 25, 24, 22, 21, 23, 20, 23, 21, 22, 20, 19, 20, 19, 20, -1, 20, 20, 23, 21, 22, 20, 19, 18, 23, 18, 20, 19, 22, 22, 21, 20, 20, 23, 23, 20, 21, 20, 23, 19, 18, 18, 19, 19, 17, 17, 19, 22, 22, 19, 18, 18, 19, 20, 21, 23, 22, 20, 23, 17, 23, 22, 20, 20, 22, 21, 18, 21, 20, 23, 21, 18, 22, 20, 19, 20, 21, 19, 21, 19, 17, 22, 21, 18, 19, 20, 21, 20, 18, 21, 18, 16, 21, 19, 19, 14, 15, 16, 16, 16, 15, 13, 14, 21, 15, 17, 15, 17, 17, 19, 16, 16, 15, 14, 12, 20, 17, 20, 22, 15, 23, 17, 13, 21, 18, 14, 23, 14, 22, 17, 22, 22, 19, 18, 21, 16, 20, 21, 17, 15, 21, 21, 14, 11, 19, 20, 20, 18, 13, 18, 12, 18, 21, 19

[25, 20, 16, 15, 14, 13, 12, 12, 12, 11, 11, 11, 11, 10, 9, 8, 7, 6, 5, 5, 6, 7, 8, 9, 10, 15, 15, 14, 14, 14, -1, 17, 16, 18, 17, 14, 14, 15, 13, 14, 11, 12, 12, 13, 12, 15, 13, 15, 15, 14, 22, 13, 17, 19, 16, 17, 18, 18, 16, 9, 18, 18, 7, 18, 16, 8, 9, 12, -1, 26, 26, 24, 24, 26, 26, 25, 25, 25, 24, 24, 24, 24, 27, 27, 27, 26, 27, 26, 25, 25, 23, 22, 19, 17, 18, 17, 21, 17, 16, 16, 17, 17, 16, 22, 15, 23, 14, 23, 17, 21, 22, 20, -1, 23, 23, 16, 14, 15, 19, 19, 21, 16, 18, 22, 22, 16, 10, 16, 16, 22, 12, 15, 15, 23, 22, 15, 21, 19, 18, 17, 17, 20, 19, 21, 14, 13, 14, 21, 20, 21, 14, 14, 13, 13, 22, 13, 19, 12, 12, 13, 22, 10, 12, 20, 12, 22, 11, 13, 16, 11, 13, 14, 15, 23, 21, 12, 22, 18, 10, 9, 13, 14, 8, 9, 11, 15, 9, 15, 19, 23, 14, 14, 17, 18, 19, 17, 19, 18, 16, 17, 9, 18, 18, 17, 14, 19, 12, 17, 17, 16, 17, 15, 8, 18, 8, 10, 16, 10, 17, 16, 9, 13, 17, 9, 15, 9, 14, 9, 8, 7, 13, 10, 13, 11, 7, 19, 14, 10, 8, 15, 14, 12, 6, 9, 12, 16, 6, 16, 6, 7, 7, 7, 9, 5, 11, 17, 6, 6, 5, 12, 

[23, 10, 5, 4, 3, 2, 1, 3, 2, 1, 0, 4, 5, 6, 7, 8, 7, 8, 9, 10, 11, 12, 13, 14, 15, 22, 22, 21, 21, 21, -1, 6, 8, 12, 16, 7, 14, 13, 15, 14, 10, 16, 16, 16, 19, 22, 2, 5, 7, 3, 15, 2, 9, 12, 8, 11, 13, 12, 10, 7, 14, 12, 5, 12, 11, 7, 19, 19, -1, 24, 24, 22, 22, 24, 24, 23, 23, 23, 22, 22, 22, 22, 25, 25, 25, 24, 25, 24, 23, 23, 21, 20, 9, 6, 8, 6, 19, 8, 5, 5, 7, 9, 6, 20, 4, 21, 3, 16, 9, 14, 15, 13, -1, 16, 16, 6, 6, 4, 12, 13, 14, 5, 10, 17, 15, 6, 2, 5, 8, 20, 1, 5, 4, 21, 20, 5, 19, 12, 12, 11, 11, 13, 11, 18, 4, 7, 5, 14, 18, 19, 6, 5, 3, 4, 20, 2, 17, 2, 3, 6, 18, 1, 4, 18, 4, 20, 3, 7, 10, 4, 7, 7, 7, 21, 19, 6, 15, 17, 5, 5, 6, 7, 4, 6, 5, 8, 3, 9, 16, 19, 8, 8, 15, 16, 17, 16, 14, 15, 16, 14, 2, 13, 15, 15, 8, 16, 6, 16, 11, 16, 15, 16, 3, 17, 6, 3, 16, 4, 11, 14, 4, 7, 13, 5, 15, 5, 8, 6, 5, 4, 8, 6, 8, 7, 6, 13, 9, 7, 7, 10, 15, 8, 5, 8, 9, 11, 6, 12, 5, 6, 6, 7, 8, 7, 9, 12, 7, 8, 6, 9, 8, 8, 8, 6, 10, 6, 9, 7, 7, 7, 9, 9, 7, 7, 11, 8, 8, 9, 10, 10, 8, 9, 8, 8, 10, 11, 10

[15, 17, 17, 16, 15, 14, 14, 18, 17, 16, 15, 17, 17, 16, 15, 16, 15, 16, 16, 16, 17, 18, 18, 17, 17, 17, 17, 16, 16, 16, -1, 16, 13, 12, 7, 8, 4, 6, 3, 5, 5, 2, 3, 3, 14, 7, 15, 13, 13, 12, 12, 13, 11, 9, 10, 10, 8, 9, 10, 15, 8, 8, 13, 9, 7, 9, 4, 10, -1, 16, 16, 14, 14, 16, 16, 15, 15, 15, 14, 14, 14, 14, 17, 17, 17, 16, 17, 16, 15, 15, 13, 12, 16, 16, 15, 15, 11, 14, 16, 15, 15, 14, 14, 12, 14, 13, 13, 13, 12, 13, 12, 13, -1, 13, 13, 14, 12, 13, 13, 12, 11, 14, 11, 12, 12, 13, 16, 12, 12, 12, 14, 14, 11, 13, 12, 14, 11, 11, 11, 11, 11, 10, 10, 11, 13, 13, 10, 11, 10, 11, 11, 12, 14, 13, 12, 14, 9, 14, 13, 11, 12, 15, 12, 10, 12, 12, 14, 12, 10, 13, 11, 10, 11, 13, 11, 12, 12, 8, 16, 15, 9, 10, 14, 15, 11, 9, 15, 9, 9, 13, 10, 10, 7, 8, 9, 7, 9, 8, 6, 7, 14, 8, 8, 7, 8, 9, 10, 7, 9, 6, 7, 5, 13, 8, 14, 14, 6, 13, 10, 6, 13, 9, 7, 12, 5, 13, 8, 11, 12, 12, 8, 10, 7, 9, 11, 10, 8, 10, 10, 8, 4, 8, 11, 9, 7, 7, 14, 6, 13, 10, 14, 15, 8, 15, 8, 8, 15, 16, 14, 6, 15, 8, 10, 14, 5, 12, 9, 

In [122]:
#combination을 구하는 함수, 가장 처음의 combination부터 가장 마지막 combination까지 구하기 위함.
@jit
def nextComb(n,S):
    find = -1
    for i in range(0,len(S)):
        if S[i] != n-len(S)+i+1:
            find = i
    if find ==-1:
        return find

    S[find] = S[find]+1

    for j in range(find+1,len(S)):
        S[j] = S[find]+j-find

    return S

In [123]:
@jit
def check(a,maximum): #위의 check와 겹치지 않게 설정
    length = len(a)
    for i in range(length):
        for j in range(i+1,length):
            first = link.loc[maximum[a[i]-1][0]]
            second = link.loc[maximum[a[j]-1][0]]
            
            #link 거리 5 안에 있음.
            check_1 = short_check[first['F_NODE']][second['F_NODE']]
            check_2 = short_check[first['F_NODE']][second['T_NODE']]
            check_3 = short_check[first['T_NODE']][second['F_NODE']]
            check_4 = short_check[first['T_NODE']][second['T_NODE']]
            
            if((check_1 and check_2) or (check_3 and check_4)):
                return 0         
    return 1

In [124]:
@jit
def Optimal_Choice2():
    value = [0 for i in range(link.shape[0])]
    
    for j,data in link.iterrows():
        sum = 0
        #교통량
        traffic = data['TRAFFIC']

        #링크의 시작노드, 끝노드
        start_node = data['F_NODE']
        end_node = data['T_NODE']

        #node의 데이터 저장
        start_node = node_sorted.loc[start_node]
        end_node = node_sorted.loc[end_node]

        #링크의 좌표는 각 node의 좌표의 평균
        coordinate = ((start_node['x좌표']+end_node['x좌표'])//2,(start_node['y좌표']+end_node['y좌표'])//2)

        #겹치는 클러스터의 수
        near = 1
        for i in range(df_cluster.shape[0]):
            if(set_cluster[i].in_distance(coordinate)):
                near += 1
        sum += traffic*near
        
        value[j] = (j,sum)
        
    return value


In [125]:
start = perf_counter()
maximum = Optimal_Choice2()
end = perf_counter()
print(end-start,'seconds')

/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/3141480079.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "Optimal_Choice2" failed type inference due to: Untyped global name 'link': Cannot determine Numba type of <class 'pandas.core.frame.DataFrame'>

File "../../../../../../../../../../var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/3141480079.py", line 3:
<source missing, REPL/exec in use?>

  @jit
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/3141480079.py:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "Optimal_Choice2" failed type inference due to: Untyped global name 'link': Cannot determine Numba type of <class 'pandas.core.frame.DataFrame'>

File "../../../../../../../../../../var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/3141480079.py", line 3:
<source missing, REPL/exec in use?>

  @jit
/opt/anac

0.5319559580002533 seconds


In [126]:
maximum.sort(key = lambda x:x[1],reverse=1)

In [127]:
maximum

[(17, 404),
 (961, 264),
 (966, 264),
 (14, 230),
 (16, 230),
 (15, 225),
 (965, 193),
 (47, 180),
 (939, 180),
 (18, 163),
 (218, 160),
 (220, 160),
 (880, 158),
 (219, 145),
 (221, 145),
 (421, 144),
 (422, 144),
 (518, 144),
 (573, 142),
 (294, 136),
 (295, 136),
 (715, 136),
 (64, 133),
 (960, 133),
 (280, 127),
 (288, 127),
 (290, 124),
 (292, 124),
 (962, 124),
 (967, 124),
 (712, 120),
 (174, 118),
 (20, 117),
 (210, 117),
 (211, 117),
 (657, 117),
 (25, 116),
 (121, 114),
 (291, 114),
 (293, 114),
 (927, 112),
 (312, 107),
 (881, 107),
 (24, 106),
 (908, 106),
 (909, 106),
 (910, 106),
 (940, 100),
 (561, 99),
 (563, 99),
 (938, 99),
 (432, 97),
 (299, 96),
 (302, 96),
 (311, 96),
 (926, 95),
 (222, 94),
 (224, 94),
 (226, 94),
 (229, 94),
 (231, 94),
 (872, 94),
 (48, 93),
 (574, 93),
 (824, 93),
 (941, 93),
 (281, 90),
 (289, 90),
 (46, 89),
 (278, 89),
 (381, 89),
 (730, 89),
 (865, 86),
 (45, 85),
 (272, 85),
 (273, 85),
 (956, 82),
 (958, 82),
 (296, 81),
 (446, 81),
 (716

In [128]:
@jit
def Optimal_Choice3():
    near_check = 50
    next = [1,2,3,4,5]

    val_maximum = [-math.inf,0]
    
    while(next!=-1):
        #for_check : 조합에 맞게 maximum에 mappingg하여 그 index를 가져옴
        for_check = []
        for i in range(len(next)):
            for_check.append(maximum[next[i]-1][0])

        #가져온 index들이 서로 인접한지 check
        if(not check(for_check,maximum)):
            next = nextComb(near_check-1,next)
            continue

        sum = 0
        for i in range(len(next)):
            sum += maximum[next[i]-1][1]

        if(sum>val_maximum[0]):
            val_maximum[0] = sum
            val_maximum[1] = for_check.copy()
            #print(val_maximum)

        next = nextComb(near_check-1,next)

    return val_maximum
    

In [129]:
result = Optimal_Choice3()

/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_2899/3407575773.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "Optimal_Choice3" failed type inference due to: No implementation of function Function(<built-in function ne>) found for signature:
 
 >>> ne(list(int64)<iv=[1, 2, 3, 4, 5]>, Literal[int](-1))
 
There are 28 candidate implementations:
      - Of which 26 did not match due to:
      Overload of function 'ne': File: <numerous>: Line N/A.
        With argument(s): '(list(int64)<iv=None>, int64)':
       No match.
      - Of which 2 did not match due to:
      Operator Overload in function 'ne': File: unknown: Line unknown.
        With argument(s): '(list(int64)<iv=None>, int64)':
       No match for registered cases:
        * (bool, bool) -> bool
        * (int8, int8) -> bool
        * (int16, int16) -> bool
        * (int32, int32) -> bool
        * (int64, int64) -> bool
        * (uint8, uint8) -> bool
  

In [131]:
result

[1216, [17, 961, 14, 218, 880]]

In [132]:
location = []
for i in result[1]:
    start = link.loc[i]['F_NODE']
    end = link.loc[i]['T_NODE']
    
    location.append((node_sorted.loc[start]['x좌표'],node_sorted.loc[start]['y좌표']))
    location.append((node_sorted.loc[end]['x좌표'],node_sorted.loc[end]['y좌표']))
    
    print(link.loc[i])

LINK_ID      1220036902
F_NODE              122
T_NODE              108
ROAD_NAME           언주로
TRAFFIC             404
LANES                 3
MAX_SPD              60
Name: 17, dtype: object
LINK_ID      1220037301
F_NODE              275
T_NODE              276
ROAD_NAME           언주로
TRAFFIC             264
LANES                 3
MAX_SPD              60
Name: 961, dtype: object
LINK_ID      1220004802
F_NODE              378
T_NODE              372
ROAD_NAME          영동대로
TRAFFIC             230
LANES                 3
MAX_SPD              60
Name: 14, dtype: object
LINK_ID      1220027203
F_NODE              225
T_NODE              228
ROAD_NAME          봉은사로
TRAFFIC             160
LANES                 3
MAX_SPD              60
Name: 218, dtype: object
LINK_ID      1220043200
F_NODE               11
T_NODE              178
ROAD_NAME           언주로
TRAFFIC             158
LANES                 2
MAX_SPD              50
Name: 880, dtype: object


In [133]:
location

[(37.4850189, 127.07381),
 (37.4825477, 127.0703225),
 (37.50268813, 127.0487588),
 (37.5029794, 127.0476938),
 (37.51660446, 127.0662259),
 (37.51598947, 127.0661311),
 (37.49494804, 127.0463903),
 (37.49559208, 127.0460863),
 (37.48903195, 127.0318923),
 (37.48993995, 127.03288)]

In [134]:
center = [37.516154,127.036135]

m = folium.Map(
    location = center,
    tiles = 'Stamen Terrain',
    zoom_start = 13
)


for num,i in enumerate(location):
        folium.Marker(i,
                      icon=folium.Icon('red', icon='star'),
                      popup = i,
                     ).add_to(m)
        print(num)
    
m

0
1
2
3
4
5
6
7
8
9
